In [2]:
!pip install torch torchvision torchaudio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.1/143.1 MB 3.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 5.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 4.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 3.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 3.8 MB/s eta 0:00:0000:0100:01m
  Obtaining dependency information for pillow!=8.3.*,>=5.3.0 from https://files.pythonhosted.org/packages/5a/29/aa1678cae507a480a6d75453c1de98940e5eb6bd8f0e8e8347ec29a4dfc0/Pillow-10.0.0-cp38-cp38-macosx_10_10_x86_64.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.2/536.2 kB 1.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 4.8 MB/s eta 0:00:0000:0100:01


In [1]:
from transformers import BertTokenizer
import torch
from gensim.models import Word2Vec
import os
import uuid
import json
import nltk

import sys
sys.path.append(os.path.join('..', 'src'))
from utils import get_sha256, clean_text, remove_non_word_chars, clean_text, tokens_to_embeddings

/Users/peterargo/anaconda3/envs/myenv/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


#### Prompt user for query
- Specify the tokenizer, consistant with Q&A model and parsed pdfs
- Prompt user for query

In [2]:
# Load the BERT tokenizer for your specific BERT model variant
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [3]:
# Prompt the user for an input query
user_query = input("Enter your query: ")

# clean query for BERT input
user_query = clean_text(user_query)

# clean query for candidate search
user_query_for_search = remove_non_word_chars(user_query)

# Tokenize the query for BERT input
tokenized_query = tokenizer.tokenize(user_query)

# Tokenize the query for candidate search
tokenized_query_for_search = tokenizer.tokenize(user_query_for_search)

# Remove the stop words for the tokenized query for search
nltk_stop_words = nltk.corpus.stopwords.words('english')
# nltk_stop_words.extend(['from', 'subject', 're', 'edu', 'use', 'table', 'figure', 'arxiv', 'sin', 'cos', 'tan', 'log', 'fx', 'ft', 'dx', 'dt', 'xt'])
tokenized_query_for_search_less_sw = [token for token in tokenized_query_for_search if token not in nltk_stop_words]

# Pad or truncate the query to a fixed length of 20 tokens (BERT input)
max_query_length = 20
if len(tokenized_query) > max_query_length:
    tokenized_query = tokenized_query[:max_query_length]
else:
    padding_length = max_query_length - len(tokenized_query)
    tokenized_query = tokenized_query + [tokenizer.pad_token] * padding_length

# Convert the tokenized query to input IDs and attention mask
input_ids_query = tokenizer.convert_tokens_to_ids(tokenized_query)
attention_mask_query = [1] * len(input_ids_query)

# Convert to tensors
input_ids_query = torch.tensor(input_ids_query).unsqueeze(0)  # Add batch dimension
attention_mask_query = torch.tensor(attention_mask_query).unsqueeze(0)  # Add batch dimension

print("Tokenized query:\n", tokenized_query, "\n")
print("Tokenized query for seach:\n", tokenized_query_for_search, "\n")
print("Tokenized query for seach less stop words:\n", tokenized_query_for_search_less_sw, "\n")
print("Input IDs query:\n", input_ids_query, "\n")
print("Attention mask query:\n", attention_mask_query, "\n")

Enter your query:  What is a contact manifold?


Tokenized query:
 ['what', 'is', 'a', 'contact', 'manifold', '?', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]'] 

Tokenized query for seach:
 ['what', 'is', 'a', 'contact', 'manifold'] 

Tokenized query for seach less stop words:
 ['contact', 'manifold'] 

Input IDs query:
 tensor([[ 2054,  2003,  1037,  3967, 19726,  1029,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0]]) 

Attention mask query:
 tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]) 



##### Add the embeddings
- Specify the embedding model

In [4]:
# Load your trained Word2Vec model
model_fname = os.path.join("..", "models", "word_embeddings", "word2vec_model.bin")
model = Word2Vec.load(model_fname)

In [16]:
# Get the query embeddings for the candidate document search
query_embeddings = tokens_to_embeddings(tokenized_query, model, RANDOM=False)
query_embeddings_search = tokens_to_embeddings(tokenized_query_for_search, model, RANDOM=False)
query_embeddings_less_sw = tokens_to_embeddings(tokenized_query_for_search_less_sw, model, RANDOM=False)

print("\t\t\t\t\t\tTokens Length\t\tEmbeddings Shape")
print(f"\t\t\t   Query embeddings:\t      {len(tokenized_query)}\t\t     {query_embeddings.shape}")
print(f"\t\tQuery embeddings for search:\t      {len(tokenized_query_for_search)}\t\t\t     {query_embeddings_search.shape}")
print(f" Query embeddings for search less stopwords:\t      {len(tokenized_query_for_search_less_sw)}\t\t\t     {query_embeddings_less_sw.shape}")

						Tokens Length		Embeddings Shape
			   Query embeddings:	      20		     (20, 100)
		Query embeddings for search:	      5			     (5, 100)
 Query embeddings for search less stopwords:	      2			     (2, 100)


##### Store the output the the query directory, filename is hash of query

In [18]:
# store the query
query_data = {
    "query": user_query,
    "input_ids_query":input_ids_query.tolist(),
    "attention_mask_query": attention_mask_query.tolist(),
    "query_search":user_query_for_search,
    "tokenized_query":tokenized_query,
    "tokenized_query_search":tokenized_query_for_search,
    "tokenized_query_search_less_sw":tokenized_query_for_search_less_sw,
    "query_embedding": query_embeddings_search.tolist(), # Just used for the candidate search
    "query_embedding_search": query_embeddings_search.tolist(), # Just used for the candidate search, cleaned
    "query_embedding_search_less_sw": query_embeddings_less_sw.tolist() # Just used for the candidate search, cleaned more
}

json_string = json.dumps(query_data['query'], indent=2)
# print(json_string)

# Specify the directory path
directory_path = os.path.join("..", 'query')

# Check if the directory exists, if not create the directory
if not os.path.exists(directory_path):
    os.makedirs(directory_path)

# Generate a UUID
# unique_id = uuid.uuid4()
unique_id = get_sha256(json_string)
print(unique_id)

fname = os.path.join(directory_path, str(unique_id)+'.json')
print(fname)

with open(fname, 'w') as j_file:
    json.dump(query_data, j_file, indent=4)

71f39bd6e1740e98a1137e7410d0bef56c40510fd934b7153c5e48428ccb0993
../query/71f39bd6e1740e98a1137e7410d0bef56c40510fd934b7153c5e48428ccb0993.json
